# Training an Article Title Generation Model with Bart

## Install libraries and download the dataset

Load kaggle.json file.

In [2]:
!pip install datasets transformers rouge-score nltk

In [3]:
!kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [4]:
!cp kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d fabiochiusano/medium-articles

Dataset URL: https://www.kaggle.com/datasets/fabiochiusano/medium-articles
License(s): CC0-1.0
 99% 364M/369M [00:13<00:00, 39.5MB/s]
100% 369M/369M [00:13<00:00, 28.3MB/s]


In [6]:
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

## Load the dataset

In [2]:
import transformers
from datasets import load_dataset, load_metric, Dataset

In [3]:
medium_datasets = load_dataset("csv", data_files="medium-articles.zip")

In [4]:
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
        num_rows: 192368
    })
})

## Dataset train/validation/test split

In [10]:
!git clone https://github.com/anukvma/group18_final_project.git

Cloning into 'group18_final_project'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 179 (delta 1), reused 4 (delta 0), pack-reused 170 (from 1)
Receiving objects: 100% (179/179), 3.39 MiB | 7.72 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [4]:
import os
import json
import pandas as pd

# Define the folder containing the text files
folder_path = '/content/group18_final_project/aiml_question_answers/AIML_QnA_Content/Group18_AIML_QA.csv'
df = pd.read_csv("/content/group18_final_project/aiml_question_answers/AIML_QnA_Content/Group18_AIML_QA.csv", names=['id','question','answer','unit'],encoding='unicode_escape',header=0)
df1 = pd.read_csv("/content/group18_final_project/aiml_question_answers/sampled_qa_data.csv", names=['id','question','answer','unit'],encoding='unicode_escape',header=0)
df = pd.concat([df, df1])


In [5]:
df.head()

,id,question,answer,unit
0,1.0,What is a linear classifier?,A linear classifier is a model that makes pred...,1.0
1,2.0,How does a linear classifier make predictions?,A linear classifier predicts by calculating th...,1.0
2,3.0,What is the objective function in a linear cla...,The objective function often used is the loss ...,1.0
3,4.0,What is gradient descent?,Gradient descent is an optimization algorithm ...,1.0
4,5.0,How does learning rate affect gradient descent?,The learning rate controls the step size in gr...,1.0


In [6]:
df.dropna(axis=0, inplace=True)

In [7]:
df.isna().sum()

,0
id,0
question,0
answer,0
unit,0


In [8]:
train_dataset: Dataset = Dataset.from_pandas(df[:800])
validation_dataset: Dataset = Dataset.from_pandas(df[800:900])
test_dataset: Dataset = Dataset.from_pandas(df[900:])

In [29]:
train_dataset

Dataset({
    features: ['id', 'question', 'answer', 'unit', '__index_level_0__'],
    num_rows: 700
})

In [9]:
# keep only a subsample of the datasets
medium_datasets["train"] = train_dataset
medium_datasets["validation"] = validation_dataset
medium_datasets["test"] = test_dataset


In [42]:
medium_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__'],
        num_rows: 227
    })
})

## Data preprocessing

In [10]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
model_checkpoint = "facebook/bart-large-xsum"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
prefix = "Answer the AIML question: "

max_input_length = 512
max_target_length = 512

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["question"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["answer"], max_length=max_target_length,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [13]:
tokenized_datasets = medium_datasets.map(preprocess_data, batched=True)
tokenized_datasets

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'unit', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

## Fine-tune T5

In [14]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!rm -r {model_dir}

rm: cannot remove '{model_dir}': No such file or directory


In [46]:
batch_size = 8
model_name = "bart-aiml-question-answer"
model_dir = f"/content/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [15]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

<ipython-input-15-a5cc15eae12d>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [49]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [50]:
trainer.train()

Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


TrainOutput(global_step=88, training_loss=2.9817764975807886, metrics={'train_runtime': 87.2663, 'train_samples_per_second': 8.021, 'train_steps_per_second': 1.008, 'total_flos': 41378153496576.0, 'train_loss': 2.9817764975807886, 'epoch': 1.0})

In [51]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


## Load the model from GDrive

In [57]:
model_name = "bart-aiml-question-answer"
model_dir = f"/content/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [53]:
text = """
What is a linear classifier?
"""

inputs = ["Answer this AIML question: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=1, max_length=512)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

A linear classifier is a type of classification algorithm that learns linear relationships between classes.


In [54]:
text = """
What is Q-Learning, and how does it work?
"""

inputs = ["Answer this AIML question: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=1, max_length=512)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Conversational AI: The Future of Customer Service

Q-Learning is a machine learning algorithm designed to solve complex data problems.


## Upload the model to the Hugging Space Hub

https://huggingface.co/docs/transformers/model_sharing

In [55]:
from huggingface_hub import notebook_login

notebook_login()

In [59]:
model.push_to_hub(repo_id="anukvma/bart-aiml-question-answer-v1")
tokenizer.push_to_hub(repo_id="anukvma/bart-aiml-question-answer-v1")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anukvma/bart-aiml-question-answer-v1/commit/f3af33cccf3b48a51cf8e821aaf47a107650cafb', commit_message='Upload tokenizer', commit_description='', oid='f3af33cccf3b48a51cf8e821aaf47a107650cafb', pr_url=None, pr_revision=None, pr_num=None)

## Load the model from the Hugging Face Hub

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')
max_input_length=512
tokenizer = AutoTokenizer.from_pretrained("anukvma/bart-aiml-question-answer-v1")
model = AutoModelForSeq2SeqLM.from_pretrained("anukvma/bart-aiml-question-answer-v1")

text = """
What is a linear classifier?
"""

inputs = ["Answer this AIML question: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=1, max_length=512)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


A linear classifier is a type of neural network that learns linear relationships between classes.


## Evaluate the model on the test set

In [17]:
import torch

# get test split
test_tokenized_dataset = tokenized_datasets["test"]

# pad texts to the same length
def preprocess_test(examples):
  inputs = [prefix + text for text in examples["question"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                           padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=8)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["answer"], max_length=100,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

{'rouge1': 27.0759,
 'rouge2': 10.1812,
 'rougeL': 21.7209,
 'rougeLsum': 23.3731,
 'gen_len': 30.6457}